### Dependencies

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import open3d as o3d
import time
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [6]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

In [47]:
vid_to_analyze = 'INSERT VIDEO PATH HERE'

In [50]:
columns = pd.MultiIndex.from_tuples([
    ('Left Hip', 'x'), ('Left Hip', 'y'), ('Left Hip', 'Angle'),
    ('Right Hip', 'x'), ('Right Hip', 'y'), ('Right Hip', 'Angle'),
    ('Left Knee', 'x'), ('Left Knee', 'y'), ('Left Knee', 'Angle'),
    ('Right Knee', 'x'), ('Right Knee', 'y'), ('Right Knee', 'Angle'),
    ('Left Ankle', 'x'), ('Left Ankle', 'y'), ('Left Ankle', 'Angle'),
    ('Right Ankle', 'x'), ('Right Ankle', 'y'), ('Right Ankle', 'Angle'),
    ('Left Foot Middle', 'x'), ('Left Foot Middle', 'y'),
    ('Right Foot Middle', 'x'), ('Right Foot Middle', 'y')
])
motion_df = pd.DataFrame(columns=columns)
motion_df.index.name = 'Frame'

frame_num = 0

cap = cv2.VideoCapture(vid_to_analyze)
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        #Recolor Image to RGB 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make Detection
        results = pose.process(image)

        #Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Image Dimensions:

        image_height = image.shape[0]
        image_width = image.shape[1]

        #Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
            left_foot = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].z]
            right_foot = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].z]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z]
            
            #Calculate Angle
            left_shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
            right_shoulder_angle = calculate_angle(right_elbow, right_shoulder, right_hip)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
            right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            left_ankle_angle = calculate_angle(left_knee, left_ankle, left_foot)
            right_ankle_angle = calculate_angle(right_knee, right_ankle, right_foot)
            left_shoulder_secondary_angle = calculate_angle(left_elbow, left_shoulder, right_shoulder)
            right_shoulder_secondary_angle = calculate_angle(right_elbow, right_shoulder, left_shoulder)
            left_hip_secondary_angle = calculate_angle(left_knee, left_hip, right_hip)
            right_hip_secondary_angle = calculate_angle(right_knee, right_hip, left_hip)

            #Calculate Foot Mid Point

            left_foot_middle = [(left_heel[0] + left_foot[0])/2, (left_heel[1] + left_foot[1])/2, (left_heel[2] + left_foot[2])/2]
            right_foot_middle = [(right_heel[0] + right_foot[0])/2, (right_heel[1] + right_foot[1])/2, (right_heel[2] + right_foot[2])/2]

            #Construct Motion Data
            row_data = {
                ('Left Hip', 'x'): left_hip[0],
                ('Left Hip', 'y'): left_hip[1],
                ('Left Hip', 'Angle'): left_hip_angle,
                ('Left Hip', 'Secondary Angle'): left_hip_secondary_angle,
                ('Right Hip', 'x'): right_hip[0],
                ('Right Hip', 'y'): right_hip[1],
                ('Right Hip', 'Angle'): right_hip_angle,
                ('Right Hip', 'Secondary Angle'): right_hip_secondary_angle,
                ('Left Knee', 'x'): left_knee[0],
                ('Left Knee', 'y'): left_knee[1],
                ('Left Knee', 'Angle'): left_knee_angle,
                ('Right Knee', 'x'): right_knee[0],
                ('Right Knee', 'y'): right_knee[1],
                ('Right Knee', 'Angle'): right_knee_angle,
                ('Left Ankle', 'x'): left_ankle[0],
                ('Left Ankle', 'y'): left_ankle[1],
                ('Left Ankle', 'Angle'): left_ankle_angle,
                ('Right Ankle', 'x'): right_ankle[0],
                ('Right Ankle', 'y'): right_ankle[1],
                ('Right Ankle', 'Angle'): right_ankle_angle,
                ('Left Foot Middle', 'x'): left_foot_middle[0],
                ('Left Foot Middle', 'y'): left_foot_middle[1],
                ('Right Foot Middle', 'x'): right_foot_middle[0],
                ('Right Foot Middle', 'y'): right_foot_middle[1]
            }

            motion_df.loc[frame_num] = row_data

            frame_num += 1



        except Exception as e:
            pass

        #Render Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )


        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


I0000 00:00:1740282250.272474   57604 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4
W0000 00:00:1740282250.321895  126886 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740282250.330925  126890 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [52]:
#Use image dimensions to un-normalize landmark data
motion_df[('Left Hip', 'x')] = motion_df[('Left Hip', 'x')]*image_width
motion_df[('Left Hip', 'y')] = motion_df[('Left Hip', 'y')]*image_height
motion_df[('Right Hip', 'x')] = motion_df[('Right Hip', 'x')]*image_width
motion_df[('Right Hip', 'y')] = motion_df[('Right Hip', 'y')]*image_height
motion_df[('Left Knee', 'x')] = motion_df[('Left Knee', 'x')]*image_width
motion_df[('Left Knee', 'y')] = motion_df[('Left Knee', 'y')]*image_height
motion_df[('Right Knee', 'x')] = motion_df[('Right Knee', 'x')]*image_width
motion_df[('Right Knee', 'y')] = motion_df[('Right Knee', 'y')]*image_height
motion_df[('Left Ankle', 'x')] = motion_df[('Left Ankle', 'x')]*image_width
motion_df[('Left Ankle', 'y')] = motion_df[('Left Ankle', 'y')]*image_height
motion_df[('Right Ankle', 'x')] = motion_df[('Right Ankle', 'x')]*image_width
motion_df[('Right Ankle', 'y')] = motion_df[('Right Ankle', 'y')]*image_height
motion_df[('Left Foot Middle', 'x')] = motion_df[('Left Foot Middle', 'x')]*image_width
motion_df[('Left Foot Middle', 'y')] = motion_df[('Left Foot Middle', 'y')]*image_height
motion_df[('Right Foot Middle', 'x')] = motion_df[('Right Foot Middle', 'x')]*image_width
motion_df[('Right Foot Middle', 'y')] = motion_df[('Right Foot Middle', 'y')]*image_height


#Set Velocity and Acceleration Columns for Each Landmark
for landmark in ['Left Hip', 'Right Hip', 'Left Knee', 'Right Knee', 'Left Ankle', 'Right Ankle']:
    angular_velocity = motion_df[(landmark, 'Angle')].diff()
    angular_acceleration = np.deg2rad(angular_velocity.diff())
    motion_df[(landmark, 'Angular Acceleration')] = angular_acceleration
for landmark in ['Left Foot Middle', 'Right Foot Middle']:
    y_col = motion_df[(landmark, 'y')]
    x_col = motion_df[(landmark, 'x')]
    velocity = np.sqrt((x_col.diff())**2 + (y_col.diff())**2)
    acceleration = velocity.diff()
    motion_df[(landmark, 'Velocity')] = velocity
    motion_df[(landmark, 'Acceleration')] = acceleration




#Interpolate Missing Values
cleaned_motion_df = motion_df.interpolate(method='linear', limit_direction='forward')
cleaned_motion_df.fillna(0, inplace=True)



In [ ]:
#Calculatte Moment at Ankles, Knees, and Hips
for side in ['Left', 'Right']:
    for joint in ['Ankle', 'Knee', 'Hip']:
        angle_col = cleaned_motion_df[(f'{side} {joint}', 'Angle')]
        angular_acceleration_col = cleaned_motion_df[(f'{side} {joint}', 'Angular Acceleration')]
        moment_col = angle_col * angular_acceleration_col
        cleaned_motion_df[(f'{side} {joint}', 'Moment')] = moment_col

In [ ]:
#Calculate Joint Moments

body_mass = 70 #Weight in KG
shank_mass = 0.0457 * body_mass
thigh_mass = 0.1447 * body_mass
foot_mass = 0.0133 * body_mass
g = 9.81 

shank_length = np.linalg.norm(np.array(cleaned_motion_df[('Left Knee', 'x')], cleaned_motion_df[('Left Knee', 'y')]) - np.array(cleaned_motion_df[('Left Ankle', 'x')], cleaned_motion_df[('Left Ankle', 'y')]))
thigh_length = np.linalg.norm(np.array(cleaned_motion_df[('Left Hip', 'x')], cleaned_motion_df[('Left Hip', 'y')]) - np.array(cleaned_motion_df[('Left Knee', 'x')], cleaned_motion_df[('Left Knee', 'y')]))
foot_length = np.linalg.norm(np.array(cleaned_motion_df[('Left Ankle', 'x')], cleaned_motion_df[('Left Ankle', 'y')]) - np.array(cleaned_motion_df[('Left Foot Middle', 'x')], cleaned_motion_df[('Left Foot Middle', 'y')]))







